In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import $ivy.`org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0`
import $ivy.`org.apache.kafka:kafka-clients:3.6.0`

In [ ]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

## Start

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [ ]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark Kafka")
                .getOrCreate()

import spark.implicits._

## Читаем из топика *input*

In [ ]:
val input = spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "localhost:29092")
      .option("subscribe", "input")
      .option("startingOffsets", "earliest")
      .load()
      .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
      .as[(String, String)]

## Пишем в топик *output*

In [ ]:
val query = input.writeStream
                    .format("kafka")
                    .option("kafka.bootstrap.servers", "localhost:29092")
                    .option("topic", "output")
                    .option("checkpointLocation", "/tmp/checkpoin")
                    .outputMode("append")
                    .start()

query.awaitTermination()

## Stop

In [ ]:
spark.stop()